In [4]:
%load_ext autoreload
%autoreload 2

from models import Detr
from utils import load_config
import torch

config = load_config(r"configs\train.yaml")

model = Detr(
    train_mode=True,
    with_gcn=True,
    pretrained='facebook/detr-resnet-50',
    checkpoint="checkpoints\encoder-pretraining-1\loss=0.86.ckpt",
    config=config)

device = torch.device('cuda:0')
model.to(device)

<>:14: SyntaxWarning: invalid escape sequence '\e'
<>:14: SyntaxWarning: invalid escape sequence '\e'
C:\Users\tangy\AppData\Local\Temp\ipykernel_16680\2112509340.py:14: SyntaxWarning: invalid escape sequence '\e'
  checkpoint="checkpoints\encoder-pretraining-1\loss=0.86.ckpt",


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

Detr(
  (model): DetrForObjectDetection(
    (model): DetrModel(
      (backbone): DetrConvModel(
        (conv_encoder): DetrConvEncoder(
          (model): FeatureListNet(
            (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            (bn1): DetrFrozenBatchNorm2d()
            (act1): ReLU(inplace=True)
            (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
            (layer1): Sequential(
              (0): Bottleneck(
                (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (bn1): DetrFrozenBatchNorm2d()
                (act1): ReLU(inplace=True)
                (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn2): DetrFrozenBatchNorm2d()
                (drop_block): Identity()
                (act2): ReLU(inplace=True)
                (aa): Identity()
                (conv3): Conv2d(64, 256

In [5]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer

# logger = WandbLogger(project='parallel_tables', name='e2e_attempt_1')

# Set up checkpointing and trainer
checkpoint_callback = ModelCheckpoint(
    monitor="validation/loss",
    filename="detr-{epoch:02d}-{validation/loss:.2f}",
    save_top_k=3,
    mode="min",
    save_last=True,
    dirpath=r"C:\Users\tangy\Downloads\DETR-GFTE\checkpoints"
)

trainer = Trainer(
    max_epochs=50,
    log_every_n_steps=10,
    logger=None,
    callbacks=[checkpoint_callback],
    val_check_interval=0.5,  # Run validation 2 times per epoch
    devices=1,
    accelerator="gpu",
    gradient_clip_val=0.1,
    accumulate_grad_batches=12
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model)

Missing logger folder: c:\Users\tangy\Downloads\DETR-GFTE\lightning_logs
c:\Users\tangy\anaconda3\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory C:\Users\tangy\Downloads\DETR-GFTE\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                   | Params | Mode 
--------------------------------------------------------------------
0 | model            | DetrForObjectDetection | 41.5 M | train
1 | simplified_tbnet | SimplifiedTbNet        | 263 K  | train
2 | gcn_criterion    | NLLLoss                | 0      | train
--------------------------------------------------------------------
41.5 M    Trainable params
222 K     Non-trainable params
41.8 M    Total params
167.063   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\tangy\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\tangy\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Batch 0 - Training Loss: 2.3237009048461914
Batch 0 - loss_ce: 0.09316719323396683
Batch 0 - loss_bbox: 0.027639105916023254
Batch 0 - loss_giou: 0.3572673499584198
Batch 0 - cardinality_error: 31.75
Batch 0 - gcn_loss: 1.3778034448623657
Batch 1 - Training Loss: 2.408313274383545
Batch 1 - loss_ce: 0.051782190799713135
Batch 1 - loss_bbox: 0.03437685966491699
Batch 1 - loss_giou: 0.4069203734397888
Batch 1 - cardinality_error: 30.75
Batch 1 - gcn_loss: 1.3708058595657349
Batch 2 - Training Loss: 1.9261085987091064
Batch 2 - loss_ce: 0.027516089379787445
Batch 2 - loss_bbox: 0.017769910395145416
Batch 2 - loss_giou: 0.23768889904022217
Batch 2 - cardinality_error: 108.0
Batch 2 - gcn_loss: 1.3343651294708252
Batch 3 - Training Loss: 2.3871939182281494
Batch 3 - loss_ce: 0.030021892860531807
Batch 3 - loss_bbox: 0.03727351874113083
Batch 3 - loss_giou: 0.40244245529174805
Batch 3 - cardinality_error: 19.5
Batch 3 - gcn_loss: 1.3659194707870483
Batch 4 - Training Loss: 1.9630613327026367

c:\Users\tangy\anaconda3\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
